In [ ]:
!pip install requests beautifulsoup4 nltk pandas openpyxl


In [ ]:
pip install jupyter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.3 MB/s eta 0:00:00


In [ ]:
# Installing necessary libraries
!pip install requests beautifulsoup4 nltk pandas openpyxl

# Importing libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import cmudict
import re
import os

# Downloading nltk resources
nltk.download('punkt')
nltk.download('cmudict')
nltk.download('stopwords')

# Loading the input Excel file
input_file = '/content/Input.xlsx'
df = pd.read_excel(input_file)

# Creating a directory to save the articles
if not os.path.exists('articles'):
    os.makedirs('articles')

# Function to extract article text from a URL
def extract_article_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        title = soup.find('h1')
        if title:
            title_text = title.get_text()
        else:
            title_text = ''

        paragraphs = soup.find_all('p')
        if not paragraphs:
            print(f"No paragraphs found in the article from {url}")
            return title_text

        article_text = ' '.join([para.get_text() for para in paragraphs])

        return title_text + '\n' + article_text
    except Exception as e:
        print(f"Failed to extract article from {url}: {e}")
        return ""

# Extracting and saving articles
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    article_text = extract_article_text(url)
    with open(f'articles/{url_id}.txt', 'w', encoding='utf-8') as file:
        file.write(article_text)

print("Articles extracted and saved.")

# Loading positive and negative words for sentiment analysis
positive_words = set(open('/content/positive-words.txt', encoding='latin1').read().split())
negative_words = set(open('/content/negative-words.txt', encoding='latin1').read().split())

# Helper functions
def syllable_count(word):
    d = cmudict.dict()
    try:
        return [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]][0]
    except KeyError:
        return 0

def complex_word(word):
    return syllable_count(word) > 2

def text_analysis(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)

    word_count = len(words)
    sentence_count = len(sentences)

    positive_score = sum(1 for word in words if word in positive_words)
    negative_score = sum(1 for word in words if word in negative_words)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (word_count + 0.000001)

    avg_sentence_length = word_count / sentence_count if sentence_count else 0
    complex_words = [word for word in words if complex_word(word)]
    percentage_complex_words = len(complex_words) / word_count if word_count else 0
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    syllables_per_word = sum(syllable_count(word) for word in words) / word_count if word_count else 0
    personal_pronouns = len(re.findall(r'\b(I|we|my|ours|us)\b', text, re.I))
    avg_word_length = sum(len(word) for word in words) / word_count if word_count else 0

    return {
        'Positive Score': positive_score,
        'Negative Score': negative_score,
        'Polarity Score': polarity_score,
        'Subjectivity Score': subjectivity_score,
        'Avg Sentence Length': avg_sentence_length,
        'Percentage of Complex Words': percentage_complex_words,
        'Fog Index': fog_index,
        'Avg Number of Words Per Sentence': avg_sentence_length,
        'Complex Word Count': len(complex_words),
        'Word Count': word_count,
        'Syllable Per Word': syllables_per_word,
        'Personal Pronouns': personal_pronouns,
        'Avg Word Length': avg_word_length
    }

# Load the articles and perform analysis
articles_dir = 'articles'
output_data = []

for index, row in df.iterrows():
    url_id = row['URL_ID']
    file_path = os.path.join(articles_dir, f'{url_id}.txt')

    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    analysis_results = text_analysis(text)
    row_data = {**row, **analysis_results}
    output_data.append(row_data)

# Saving the results to an Excel file
output_df = pd.DataFrame(output_data)
output_df.to_excel('Output Data Structure.xlsx', index=False)

print("Text analysis completed and saved.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Articles extracted and saved.


In [ ]:
# Install necessary libraries
!pip install requests beautifulsoup4 nltk pandas openpyxl

# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import cmudict
import re
import os

# Download nltk resources
nltk.download('punkt')
nltk.download('cmudict')
nltk.download('stopwords')

# Load the input Excel file
input_file = '/content/Input.xlsx'
df = pd.read_excel(input_file)

# Create a directory to save the articles
if not os.path.exists('articles'):
    os.makedirs('articles')

# Function to extract article text from a URL
def extract_article_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        title = soup.find('h1')
        if title:
            title_text = title.get_text()
        else:
            title_text = ''

        paragraphs = soup.find_all('p')
        if not paragraphs:
            print(f"No paragraphs found in the article from {url}")
            return title_text

        article_text = ' '.join([para.get_text() for para in paragraphs])

        return title_text + '\n' + article_text
    except Exception as e:
        print(f"Failed to extract article from {url}: {e}")
        return ""

# Extract and save articles
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    article_text = extract_article_text(url)
    with open(f'articles/{url_id}.txt', 'w', encoding='utf-8') as file:
        file.write(article_text)

print("Articles extracted and saved.")

# Load positive and negative words for sentiment analysis
positive_words = set(open('/content/positive-words.txt', encoding='latin1').read().split())
negative_words = set(open('/content/negative-words.txt', encoding='latin1').read().split())

# Helper functions
def syllable_count(word):
    d = cmudict.dict()
    try:
        return [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]][0]
    except KeyError:
        return 0

def complex_word(word):
    return syllable_count(word) > 2

def text_analysis(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)

    word_count = len(words)
    sentence_count = len(sentences)

    positive_score = sum(1 for word in words if word in positive_words)
    negative_score = sum(1 for word in words if word in negative_words)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (word_count + 0.000001)

    avg_sentence_length = word_count / sentence_count if sentence_count else 0
    complex_words = [word for word in words if complex_word(word)]
    percentage_complex_words = len(complex_words) / word_count if word_count else 0
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    syllables_per_word = sum(syllable_count(word) for word in words) / word_count if word_count else 0
    personal_pronouns = len(re.findall(r'\b(I|we|my|ours|us)\b', text, re.I))
    avg_word_length = sum(len(word) for word in words) / word_count if word_count else 0

    return {
        'Positive Score': positive_score,
        'Negative Score': negative_score,
        'Polarity Score': polarity_score,
        'Subjectivity Score': subjectivity_score,
        'Avg Sentence Length': avg_sentence_length,
        'Percentage of Complex Words': percentage_complex_words,
        'Fog Index': fog_index,
        'Avg Number of Words Per Sentence': avg_sentence_length,
        'Complex Word Count': len(complex_words),
        'Word Count': word_count,
        'Syllable Per Word': syllables_per_word,
        'Personal Pronouns': personal_pronouns,
        'Avg Word Length': avg_word_length
    }

# Load the articles and perform analysis
articles_dir = 'articles'
output_data = []

for index, row in df.iterrows():
    url_id = row['URL_ID']
    file_path = os.path.join(articles_dir, f'{url_id}.txt')

    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    analysis_results = text_analysis(text)
    row_data = {**row, **analysis_results}
    output_data.append(row_data)

# Save the results to an Excel file
output_df = pd.DataFrame(output_data)
output_df.to_excel('Output Data Structure.xlsx', index=False)

print("Text analysis completed and saved.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Articles extracted and saved.
